In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [26]:
# load all the saved files

model = load_model('model.keras')

with open('data_transformers/gender_encoder.pkl', 'rb') as file:
    gender_encoder = pickle.load(file)
    
with open('data_transformers/geo_encoder.pkl', 'rb') as file:
    geo_encoder = pickle.load(file)
    
with open('data_transformers/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
    
    

In [30]:
input_data = {
'CreditScore': 600,
'Geography': 'France',
'Gender': 'Male',
'Age': 40,
'Tenure': 3,
'Balance': 60000,
'NumOfProducts': 2,
'HasCrCard': 1,
'IsActiveMember': 1,
'EstimatedSalary': 50000
}

In [31]:
data = pd.DataFrame([input_data])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [33]:
data.Gender = gender_encoder.transform(data.Gender)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [32]:
geo_encoded = geo_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geo_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Code\DataScience\Churn-prediction\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [35]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [36]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [38]:
data = scaler.transform(data)
data

array([[-0.53068769,  0.9104886 ,  0.10735072, -0.68625772, -0.26789121,
         0.81366744,  0.64628603,  0.96704278, -0.87443538,  0.99575899,
        -0.57985213, -0.57157662]])

In [39]:
pred = model.predict(data)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


array([[0.02939199]], dtype=float32)

In [40]:
probablity = pred[0][0]
probablity

np.float32(0.029391985)

In [41]:
if probablity >= 0.5 :
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn. ')

The customer is not likely to churn. 
